In [1]:
import pandas as pd

In [2]:
df_csv = pd.read_csv("dataset/Crimes_-_2023_20250508.zip")

df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262950 entries, 0 to 262949
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ID                    262950 non-null  int64  
 1   Case Number           262950 non-null  object 
 2   Date                  262950 non-null  object 
 3   Block                 262950 non-null  object 
 4   IUCR                  262950 non-null  object 
 5   Primary Type          262950 non-null  object 
 6   Description           262950 non-null  object 
 7   Location Description  261512 non-null  object 
 8   Arrest                262950 non-null  bool   
 9   Domestic              262950 non-null  bool   
 10  Beat                  262950 non-null  int64  
 11  District              262950 non-null  int64  
 12  Ward                  262947 non-null  float64
 13  Community Area        262950 non-null  int64  
 14  FBI Code              262950 non-null  object 
 15  

In [3]:
print(df_csv["Year"].unique())

[2023]
